In [2]:
import requests
import pandas as pd
import time
import os
import sys

# Add the 'scripts' folder to the Python path
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from discord_notifier import send_discord_message

# Ensure the directory exists
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to insert summoner data batch into a CSV file
def insert_summoner_data_batch(summoner_data_batch, csv_file='../data/processed/summoner_details.csv'):
    # Ensure the directory exists
    ensure_directory_exists(os.path.dirname(csv_file))
    
    # Check if the file exists
    if os.path.exists(csv_file):
        df_summoner_details = pd.read_csv(csv_file)
    else:
        # If it doesn't exist, create an empty DataFrame with appropriate columns
        df_summoner_details = pd.DataFrame(columns=[
            'summoner_id', 'account_id', 'puuid', 'profile_icon_id', 'revision_date', 'summoner_level'
        ])

    # Convert the summoner_data_batch into a DataFrame
    new_data = pd.DataFrame(summoner_data_batch, columns=df_summoner_details.columns)

    # Append new data and remove duplicates based on 'summoner_id'
    df_summoner_details = pd.concat([df_summoner_details, new_data]).drop_duplicates(subset=['summoner_id'])

    # Save the updated DataFrame back to CSV
    df_summoner_details.to_csv(csv_file, index=False)
    print(f"Inserted {len(new_data)} summoner entries into {csv_file}.")

# Function to pull summoner data from the API and insert it into the CSV file
def pull_summoner_data():
    api_key = "RGAPI-b8762239-29e3-48eb-a6c7-5b9198da9aa1"
    base_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/"
    summoner_data_batch = []

    # Read summoner IDs from entries.csv that are not yet in summoner_details.csv
    if os.path.exists('../data/processed/entries.csv'):
        df_entries = pd.read_csv('../data/processed/entries.csv')
    else:
        print("No entries.csv found.")
        return
    
    if os.path.exists('../data/processed/summoner_details.csv'):
        df_summoner_details = pd.read_csv('../data/processed/summoner_details.csv')
        existing_summoner_ids = df_summoner_details['summoner_id'].unique()
        summoner_ids_to_fetch = df_entries[~df_entries['summoner_id'].isin(existing_summoner_ids)]['summoner_id'].tolist()
    else:
        summoner_ids_to_fetch = df_entries['summoner_id'].tolist()

    print(f"values that need to be pulled from the Riot API: {len(summoner_ids_to_fetch)}")

    # Iterate over summoner IDs and pull data one by one
    count = 1
    total = len(summoner_ids_to_fetch)
    for summoner_id in summoner_ids_to_fetch:
        
        print(f"Fetching values for: {summoner_id}:  {count}/{total}")
        url = f"{base_url}{summoner_id}?api_key={api_key}"
        response = requests.get(url)
        if response.status_code != 200:
            print("Error with the request:", response.status_code)
            continue
        
        summoner_data = response.json()
        summoner_data_batch.append((
            summoner_data['id'], summoner_data['accountId'], summoner_data['puuid'], 
            summoner_data['profileIconId'], summoner_data['revisionDate'], summoner_data['summonerLevel']
        ))

        # Respect the rate limit
        count += 1
        time.sleep(1.3)

    # Insert all fetched data into the CSV file in a single batch
    if summoner_data_batch:
        print('Inserting values into CSV')
        insert_summoner_data_batch(summoner_data_batch)

if __name__ == "__main__":
    try:
        pull_summoner_data()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        send_discord_message()


values that need to be pulled from the Riot API: 24
Fetching values for: 1gpcsptEJ1ygv615JJ34OXV_BrB7kM29bj4dcujEjpsf:  1/24
Error with the request: 403
Fetching values for: vTZED1RM0dpcr1d9Xe5qS3TqJ7QiYhGPyUw_y4xKd2qqyDI:  1/24
Error with the request: 403
Fetching values for: Cku1lrcFKLTrqUhJDo7jJXEWeQ5YsUlr9TfXXOdOiT5glaYl:  1/24
Error with the request: 403
Fetching values for: TBnBgx-u0fXjmY4ksD8hRIBH5s_yf1A0xm1OZzLpETXn:  1/24
Error with the request: 403
Fetching values for: 5Mw9gFSDnfvCOtQH-mMOpsrHg5f-8KnU5z88CdHirFQHYv4:  1/24
Error with the request: 403
Fetching values for: 8l8fWz4QMav7NH2l2A12dE7D1S_k-5zUZgxM9-vqDbQMsUA:  1/24
Fetching values for: rAo605YYa1ik5mhP35awnP8_Dtaumm1LuyUHAgAZGPMe5H7r:  2/24
Error with the request: 403
Fetching values for: hKEZK0BcRbZuKi_jDw8vHAJMuybJAOBoH1Km5Tk5BNilKktl:  2/24
Fetching values for: wwpgOwaMA2bAKt0kS1f4qDdwyjYnTh8eO9rY-pJBBp-LJQfs:  3/24
Fetching values for: 6pCQlf0ByQgCjs1E9-Cv7zO6ApcqpYySoOzqr0wA5C3CAz4:  4/24
Fetching values for: l